In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0,'/home/prashant/Imputer.py/imputer')

In [144]:
train = pd.read_csv('./Train.csv')
test = pd.read_csv('./Test.csv')

In [145]:
train['Item_Type_Combined'] = train['Item_Identifier'].apply(lambda x: x[0:2])
train['Item_Type_Combined'] = train['Item_Type_Combined'].map({'FD':'Food','NC':'Non-Consumable','DR':'Drink'})
train.loc[train['Item_Type_Combined']=="Non-Consumable",'Item_Fat_Content'] = "Non-Edible"
test['Item_Type_Combined'] = test['Item_Identifier'].apply(lambda x: x[0:2])
test['Item_Type_Combined'] = test['Item_Type_Combined'].map({'FD':'Food','NC':'Non-Consumable','DR':'Drink'})
test.loc[test['Item_Type_Combined']=="Non-Consumable",'Item_Fat_Content'] = "Non-Edible"
combine = [train,test]
type_dict=list(train['Item_Type'].value_counts().index)
type_item = {}
i=1
for item in type_dict:
    type_item[item] = i
    i=i+1

    
outlet=list(train['Outlet_Size'].value_counts().index)    
outlet_size = {}
i=1
for item in outlet:
    outlet_size[item] = i
    i=i+1


outlet_identifier=list(train['Outlet_Identifier'].value_counts().index)
i=1
identifier = {}
for item in outlet_identifier:
    identifier[item] = i
    i=i+1




outlet_location = list(train['Outlet_Location_Type'].value_counts().index)
i=1
location = {}
for item in outlet_location:
    location[item] = i
    i=i+1


    
outlet_type = list(train['Outlet_Type'].value_counts().index)
i=1
type1 ={}
for item in outlet_type:
    type1[item] = i
    i=i+1

item_id = list(train['Item_Identifier'].value_counts().index)
i=1
id_dict ={}
for item in item_id:
    id_dict[item] = i
    i=i+1

abc = list(train['Item_Type_Combined'].value_counts().index)
i=1
abc1 ={}
for item in abc:
    abc1[item] = i
    i=i+1
    
for dataset in combine:
    dataset['Item_Identifier'] = dataset['Item_Identifier'].map(id_dict)
    dataset['Item_Fat_Content']=dataset['Item_Fat_Content'].replace(['Low Fat','LF','low fat'], 1)
    dataset['Item_Fat_Content']=dataset['Item_Fat_Content'].replace(['Regular','reg'], 2)
    dataset['Item_Fat_Content']=dataset['Item_Fat_Content'].replace(['Non-Edible'], 2)
    dataset['Item_Type'] = dataset['Item_Type'].map(type_item)
    dataset['Outlet_Identifier'] = dataset['Outlet_Identifier'].map(identifier)
    dataset['Outlet_Size'] = dataset['Outlet_Size'].map(outlet_size)
    dataset['Outlet_Location_Type'] = dataset['Outlet_Location_Type'].map(location)
    dataset['Outlet_Type'] = dataset['Outlet_Type'].map(type1)
    dataset['Item_Type_Combined'] = dataset['Item_Type_Combined'].map(abc1)
    

In [146]:
from imputer import Imputer
impute = Imputer()


In [147]:
columns = list(train.columns)
train_imputed = impute.knn(X=train, column='Item_Weight', k=10)
train = pd.DataFrame(train_imputed)
train.columns = columns
train_imputed = impute.knn(X=train, column='Outlet_Size', k=10)

/home/prashant/Imputer.py/imputer/imputer.py:118: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  X = X.as_matrix()


In [148]:
train = pd.DataFrame(train_imputed)
train.columns = columns

In [149]:
for dataset in combine:
    dataset = dataset.drop(columns=['Item_Outlet_Sales','Item_Type','Outlet_Establishment_Year'])

In [150]:
test = test.drop(columns=['Item_Identifier'])

In [151]:
columns = list(test.columns)
test_imputed = impute.knn(X=test, column='Item_Weight', k=10)
test = pd.DataFrame(test_imputed)
test.columns = columns
test_imputed = impute.knn(X=test, column='Outlet_Size', k=10)
test = pd.DataFrame(test_imputed)
test.columns = columns

/home/prashant/Imputer.py/imputer/imputer.py:118: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  X = X.as_matrix()


In [152]:
columns = list(train.columns)
labels = ['Item_Outlet_Sales']
columns.remove('Item_Outlet_Sales')
X = train[columns]
y = train[labels]

In [153]:
from sklearn.model_selection import train_test_split

In [154]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42 )

In [187]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor()
model.fit(X_train, y_train)
predict = model.predict(X_test)
ans=0
for i in range(0,len(y_test.values)):
    ans=ans + abs((y_test.values[i][0]-predict[i])/y_test.values[i][0])
y_true, y_pred = np.array(y_test), np.array(predict)
np.mean(np.abs((y_true - y_pred) / y_true)) * 100

260.9028588180342

In [186]:
from sklearn.linear_model import LinearRegression
model = LinearRegression(normalize=True)
model.fit(X_train, y_train)
predict = model.predict(X_test)
ans=0
for i in range(0,len(y_test.values)):
    ans=ans + abs((y_test.values[i][0]-predict[i])/y_test.values[i][0])
y_true, y_pred = np.array(y_test), np.array(predict)
np.mean(np.abs((y_true - y_pred) / y_true)) * 100

106.79094930548673

In [185]:
from sklearn.ensemble import RandomForestRegressor
model= RandomForestRegressor(n_estimators=400,max_depth=6, min_samples_leaf=100,n_jobs=4,)
model.fit(X_train, y_train)
predict = model.predict(X_test)
ans=0
for i in range(0,len(y_test.values)):
    ans=ans + abs((y_test.values[i][0]-predict[i])/y_test.values[i][0])

y_true, y_pred = np.array(y_test), np.array(predict)
np.mean(np.abs((y_true - y_pred) / y_true)) * 100

/home/prashant/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


249.2863138133714

In [189]:
y_true, y_pred = np.array(y_test), np.array(predict)
np.mean(np.abs((y_true - y_pred) / y_true)) * 100

260.9028588180342

In [173]:
target = 'Item_Outlet_Sales'
IDcol = ['Item_Identifier','Outlet_Identifier']
mean_sales = train['Item_Outlet_Sales'].mean()
from sklearn import cross_validation, metrics
def modelfit(alg, dtrain, dtest, predictors, target, IDcol, filename):
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain[target])
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])

    #Perform cross-validation:
    cv_score = cross_validation.cross_val_score(alg, dtrain[predictors], dtrain[target], cv=20, scoring='mean_squared_error')
    cv_score = np.sqrt(np.abs(cv_score))
    
    #Print model report:
    print ("\nModel Report")
    print ("RMSE : %.4g" % np.sqrt(metrics.mean_squared_error(dtrain[target].values, dtrain_predictions)))
    print ("CV Score : Mean - %.4g | Std - %.4g | Min - %.4g | Max - %.4g" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score)))
    
    #Predict on testing data:
    dtest[target] = alg.predict(dtest[predictors])
    
    #Export submission file:
    IDcol.append(target)
    submission = pd.DataFrame({ x: dtest[x] for x in IDcol})
    submission.to_csv(filename, index=False)

/home/prashant/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [181]:
len(y_test)

2813